In [29]:
import numpy as np
import pandas as pd
import sqlite3
import sys
import os
import re
from urllib.parse import urlparse

In [83]:
def get_content_type_from_headers(x):
    ls = x.replace('"','').split('],[')
    for l in ls:
        ta = l.replace(']]','').split(',')
        if "content-type" in ta:
            return ta[-1]
    return ''
def get_content_length_from_headers(x):
    ls = x.replace('"','').split('],[')
    for l in ls:
        ta = l.replace(']]','').split(',')
        if "content-length" in ta:
            return int(ta[-1])
    return 0
def extract_host_from_url(url_ls):
    return list(map(lambda x: urlparse(x).netloc.split('.')[1] if len(urlparse(x).netloc.split('.')) > 1 else urlparse(x).netloc, url_ls))
def is_third_party(origin, url_host):
    if origin in url_host:
        return False
    return True
def extract_deep_clean_host(url):
    s = url.strip(".au").strip('.uk')
    return s.split('.')[-2] if len(s.split('.')) > 1 else s
def save_df_to_csv(df, file_name):
    df.to_csv(file_name+'.csv', index=False)

In [9]:
data_path = "exp-data/"
type_name = 'forbes'
exp_type= 'c'
file_name = type_name + exp_type
sql_file = data_path + file_name + '.sqlite'

In [10]:
query = """SELECT sv.site_url, sv.visit_id,
        hr.url, hr.headers
        FROM http_responses as hr LEFT JOIN site_visits as sv
        ON sv.visit_id = hr.visit_id
        """

In [86]:
def extract_tracker_table(type_name, exp_type, num):
    sql_file = data_path + type_name + "-" + exp_type + str(num) + '.sqlite'
    print(sql_file)
    conn = sqlite3.connect(sql_file)
    http_responses = pd.read_sql_query(query, conn)

    http_responses['origin_site'] = extract_host_from_url(http_responses.site_url)
    http_responses["url_host"] = list(map(lambda x:urlparse(x).netloc, http_responses.url))
    http_responses["content_type"] = list(map(lambda x:get_content_type_from_headers(x), http_responses.headers))
    http_responses["content_length"] = list(map(lambda x:get_content_length_from_headers(x), http_responses.headers))
    http_responses["url_host_name"] = list(map(lambda x:extract_deep_clean_host(x), http_responses.url_host))

    http_responses['is_third_party'] = list(map(lambda a, b: is_third_party(a, b), http_responses.origin_site, http_responses.url_host))

    http_responses_images = http_responses[http_responses.content_type.str.contains('image')]
    http_responses_images_trackers = http_responses_images[http_responses_images.content_length < 100000]

    http_responses_images_trackers = http_responses_images_trackers[['origin_site', 'url_host', 'url_host_name','content_type', 'content_length', 'is_third_party']]

    df = http_responses_images_trackers.groupby(['origin_site', 'url_host_name']).size().reset_index(name='count')
    df['exp_type'] = [type_name] * df.shape[0]
    df['exp_num'] = [exp_type+str(num)] * df.shape[0]
    return df

In [87]:
types = ['nyt', 'forbes', 'washington']
exps = ['t', 'c']
df_ls = []
for i in range(1, 6):
    for t in types:
        for e in exps:
            df = extract_tracker_table(t, e, i)
            df_ls.append(df)

exp-data/nyt-t1.sqlite
exp-data/nyt-c1.sqlite
exp-data/forbes-t1.sqlite
exp-data/forbes-c1.sqlite
exp-data/washington-t1.sqlite
exp-data/washington-c1.sqlite
exp-data/nyt-t2.sqlite
exp-data/nyt-c2.sqlite
exp-data/forbes-t2.sqlite
exp-data/forbes-c2.sqlite
exp-data/washington-t2.sqlite
exp-data/washington-c2.sqlite
exp-data/nyt-t3.sqlite
exp-data/nyt-c3.sqlite
exp-data/forbes-t3.sqlite
exp-data/forbes-c3.sqlite
exp-data/washington-t3.sqlite
exp-data/washington-c3.sqlite
exp-data/nyt-t4.sqlite
exp-data/nyt-c4.sqlite
exp-data/forbes-t4.sqlite
exp-data/forbes-c4.sqlite
exp-data/washington-t4.sqlite
exp-data/washington-c4.sqlite
exp-data/nyt-t5.sqlite
exp-data/nyt-c5.sqlite
exp-data/forbes-t5.sqlite
exp-data/forbes-c5.sqlite
exp-data/washington-t5.sqlite
exp-data/washington-c5.sqlite


In [88]:
result = pd.concat(df_ls)

In [98]:
save_df_to_csv(result, 'image_trakers_count')

In [103]:
trackers_t = result[result.exp_num.str.contains('t')].groupby(['exp_type', 'origin_site', 'url_host_name'])['count'].sum().reset_index(name='sum')

In [104]:
trackers_c = result[result.exp_num.str.contains('c')].groupby(['exp_type', 'origin_site', 'url_host_name'])['count'].sum().reset_index(name='sum')

In [114]:
combined_trackers = pd.merge(trackers_c, trackers_t, on=['exp_type','origin_site', 'url_host_name'], how='outer', suffixes=('_control', '_treatment'))

# trackers_t.join(trackers_c, on=['exp_type','origin_site', 'url_host_name'],  how='outer')

In [124]:
combined_trackers = combined_trackers.fillna(0)

In [126]:
combined_trackers[combined_trackers.sum_control < combined_trackers.sum_treatment].shape

(166, 5)

In [127]:
combined_trackers[combined_trackers.sum_control > combined_trackers.sum_treatment].shape

(84, 5)

In [128]:
combined_trackers[combined_trackers.sum_control == combined_trackers.sum_treatment].shape

(31, 5)

In [129]:
save_df_to_csv(combined_trackers, 'image_trakers_combined')

In [143]:
result['exp_group'] = list(map(lambda x: x[0], result.exp_num))
result['exp_index'] = list(map(lambda x: x[1], result.exp_num))

In [138]:
combined_trackers.groupby(['exp_type', 'origin_site'])[['sum_control', 'sum_treatment']].sum().stack().reset_index()

,exp_type,origin_site,level_2,0
0,forbes,forbes,sum_control,438.0
1,forbes,forbes,sum_treatment,454.0
2,forbes,mayoclinic,sum_control,661.0
3,forbes,mayoclinic,sum_treatment,1099.0
4,forbes,webmd,sum_control,2035.0
5,forbes,webmd,sum_treatment,1793.0
6,nyt,mayoclinic,sum_control,640.0
7,nyt,mayoclinic,sum_treatment,1233.0
8,nyt,nytimes,sum_control,495.0
9,nyt,nytimes,sum_treatment,537.0


In [146]:
for_model = result.groupby(['exp_index', 'exp_group', 'exp_type', 'origin_site'])['count'].sum().reset_index(name='sum')

In [147]:
save_df_to_csv(for_model, "pixel_tracker_for_model")